In [10]:
import csv
import os
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

# 크롬 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 필요시 주석 처리
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# 기본 설정
base_url = 'http://www.kanauction.kr/auction/past/main'
filename = 'auction_data.csv'

# ===== 1. 기존 CSV에서 중복 키 불러오기 =====
existing_keys = set()
if os.path.exists(filename):
    with open(filename, 'r', encoding='utf-8-sig') as f:
        reader = csv.reader(f)
        next(reader)  # 헤더 생략
        for row in reader:
            if len(row) >= 3:
                key = f"{row[0]}|{row[1]}|{row[2]}"
                existing_keys.add(key)

# ===== 2. 마지막 페이지 감지 =====
driver.get(base_url)
time.sleep(2)
paging_last = driver.find_element(By.CSS_SELECTOR, 'span.pagingLast > a')
last_href = paging_last.get_attribute('href')
last_page_match = re.search(r'page=(\d+)', last_href)
last_page = int(last_page_match.group(1)) if last_page_match else 1

print(f"[✔] 감지된 마지막 페이지: {last_page}페이지")

# ===== 3. CSV 열고 한 줄씩 즉시 저장 =====
write_header = not os.path.exists(filename)
with open(filename, 'a', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(['작가명', '경매일', '작품정보', '추정가', '낙찰가', '차액'])

    for page in range(1, last_page + 1):
        print(f"\n======= [페이지 {page}] =======\n")
        driver.get(f"{base_url}?page={page}")
        time.sleep(2)

        try:
            container = driver.find_element(By.ID, 'articleListMainBox')
            items = container.find_elements(By.CSS_SELECTOR, 'li.info.breakWord')
        except:
            print(f"[!] 페이지 {page}에서 항목을 찾을 수 없습니다.")
            continue

        for idx, item in enumerate(items):
            try:
                artist = item.find_element(By.CSS_SELECTOR, 'span.title').text.strip()
                auc_date = item.find_element(By.CSS_SELECTOR, 'span.aucDate').text.strip()
                auc_title = item.find_element(By.CSS_SELECTOR, 'span.aucTitle').text.strip()
                key = f"{artist}|{auc_date}|{auc_title}"

                if key in existing_keys:
                    print(f"[🔁] 중복 - 건너뜀: {key}")
                    continue

                price_tags = item.find_elements(By.CSS_SELECTOR, 'span.price')
                est_price_text = price_tags[0].text.replace("추정가 KRW", "").strip()
                est_low = re.findall(r'[\d,]+', est_price_text)[0].replace(",", "")
                est_low_int = int(est_low)

                if len(price_tags) > 1 and "낙찰가" in price_tags[1].text:
                    sold_price_text = price_tags[1].text.replace("낙찰가 KRW", "").strip()
                    sold_price_clean = re.sub(r'[^\d]', '', sold_price_text)
                    sold_price_int = int(sold_price_clean)
                    diff = sold_price_int - est_low_int
                    sold_price_display = f"{sold_price_int:,}"
                    diff_display = f"{diff:,}"
                else:
                    sold_price_display = "낙찰 X"
                    diff_display = "N/A"

                # 저장
                writer.writerow([
                    artist, auc_date, auc_title,
                    est_price_text, sold_price_display, diff_display
                ])
                f.flush()  # 시스템 충돌 대비 저장 강제 반영
                existing_keys.add(key)

                print(f"✅ 저장 완료: {artist} / {auc_date}")

            except Exception as e:
                print(f"[!] 항목 오류 발생 ({idx+1}): {e}")

driver.quit()
print(f"\n[📁] 크롤링 완료. 데이터는 '{filename}'에 저장됐습니다.")


[✔] 감지된 마지막 페이지: 519페이지

======= [페이지 1] =======

[🔁] 중복 - 건너뜀: 유희강 柳熙綱|2025.02.20|영수가복 永受嘉福 | 종이에 먹 | 32x128cm | 진주 晋州
[🔁] 중복 - 건너뜀: 김충현 金忠顯|2025.02.20|심정흥장 心靜興長 | 1996년(병자) | 종이에 먹 | 34x136cm | 안동 安東/서경 恕卿
[🔁] 중복 - 건너뜀: 김응현 金膺顯 서, 방철주 方澈柱 도자|2025.02.20|용비어천가 龍飛御天歌 제 2장 | 1976년(병인) | 청자에 안료 | 28.4x6.2x11cm | 안동 安東/선경 善卿
[🔁] 중복 - 건너뜀: 유희강 柳熙綱|2025.02.20|반곡 盤谷·망천 輞川 대련 | 종이에 먹 | 128.6x21.5cmx2 | 진주 晋州
[🔁] 중복 - 건너뜀: 퇴경 상로 退耕 相老|2025.02.20|각지일표래 各持一瓢來 | 종이에 먹 | 14.5x50cm | 안동 安東
[🔁] 중복 - 건너뜀: 경봉 정석 鏡峰 靖錫|2025.02.20|서산대사 선시 西山大師 禪詩 | 종이에 먹 | 32x42.3cm | 광주 光州
[🔁] 중복 - 건너뜀: 이철경 李喆卿|2025.02.20|그리운 날 : 가람 이병기 嘉藍 李秉岐 시 | 종이에 먹 | 90.7x33.2cm | 한산 韓山
[🔁] 중복 - 건너뜀: 서희환 徐喜煥|2025.02.20|. | 1986년 | 종이에 먹 | 125.2x30.6cm | 이천利川
[🔁] 중복 - 건너뜀: 이갑성 李甲成|2025.02.20|보산정 寶山亭 | 1958년(무술) | 종이에 먹 | 39.5x101.7cm | 경주 慶州
[🔁] 중복 - 건너뜀: 서희환 徐喜煥|2025.02.20|보람 | 1988년 | 종이에 먹 | 33.2x61.4cm | 이천 利川

======= [페이지 2] =======

[🔁] 중복 - 건너뜀: 유희강 柳熙綱|2025.02.20|답사마승정상검경 答司馬承禎上劍鏡 | 종이에 먹 | 132x39.5cm | 진주 晋州
[🔁] 중복 - 건너뜀: 